In [1]:
#reading in the verdict
with open("The_verdict.txt","r",encoding="utf-8") as f:
    #using file.read() method to read in the entire text.
    raw_text = f.read()
print("Total number of chars:", len(raw_text))
print(raw_text[:99])


Total number of chars: 20480
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
#Split the text on whitespace chars
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [3]:
#creating a set of all entries
all_tokens = sorted(list(set(preprocessed)))
size = len(all_tokens)
all_tokens.extend(["<|endoftext|>","<unk>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(all_tokens))

1132


In [4]:
class SimpleTokenizer:
    def __init__(self, vocab: dict):
        #Creating a field to store the dict
        self.str_to_int = vocab
        #Another dict field to reverse the dict
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text: str) -> list[int]:
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        #This cleans the pre processed off the empty strings !
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        #Creating a list of ids
        
        preprocessed = [item if item in self.str_to_int else "<unk>" for item in preprocessed]
        
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self,ids:list[int]) -> str:
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [5]:
# Instantiating an object to take care of this !
tokenize = SimpleTokenizer(vocab)
text = "It's the last he painted, you know"
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<unk>', 1131)


In [6]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))

In [7]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [8]:
import tiktoken as tk
tokenize=tk.get_encoding("gpt2")

In [9]:
integers = tokenize.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 20562, 13]


In [10]:
hahaha = tokenize.encode("buhahahaha")
unknown = tokenize.decode(hahaha)
print(hahaha)

[65, 7456, 36225, 12236]


In [11]:
#Tokenising the book verdict
with open("The_verdict.txt", "r", encoding="utf-8") as fi:
    raw_text = fi.read()

enc_text = tokenize.encode(raw_text)
enc_sample = enc_text[50:]

In [12]:
#Creating the input target pairs
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x:{x}")
print(f"y:     {y}")

x:[290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [13]:
import torch
#For all i in range of 1 to the context size of 4
for i in range(1, context_size):
    #The context is the first i-1 digits
    context = enc_sample[:i]
    desired = enc_sample[i]
    #One word tokens should always be converted to arrays..
    print(tokenize.decode(context), "---->", tokenize.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in


In [14]:
#Import pytorch torch library
import torch
#From torch utils data import the Dataset and DataLoader class
from torch.utils.data import Dataset, DataLoader

#The class GPTDatasetV1 inherits from the Dataset class of torch
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length,stride):
            input_chunk = token_ids[i:i + max_length]
            #The target chunk is to replicate the way GPT receives the target (the word prediction)
            target_chunk = token_ids[i+1 : i + max_length + 1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

    

In [15]:
#The dataloader method has been created to access the data set class and use the 
#same to create the batches with input pairs
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tk.get_encoding("gpt2")                         #1
    
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)   #2
    #Data loader is an instance of the torch DataLoader class
    dataloader = DataLoader(
        #It needs to have a dataset
        dataset,
        # A batch size
        batch_size=batch_size,
        # Shuffle if req
        shuffle=shuffle,
        # Drop last option
        drop_last=drop_last,     #3
        # And number of workers
        num_workers=num_workers     #4
    )

    return dataloader

In [16]:
with open("The_Verdict.txt","r",encoding="utf-8") as fi:
    raw_text = fi.read()

dataLoader = create_dataloader_v1(raw_text, batch_size=4, max_length=4, stride=4, shuffle=False)
#The iter method gets the iterator for the data loader
data_iter = iter(dataLoader)
#The next method runs the iterator to get the 1st batch from the iteration
first_batch = next(data_iter)
#Lets try 2nd
second = next(data_iter)

In [17]:
#Trying the token embeddings

input_ids = torch.tensor([2,3,5,1])
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embediing_layer = torch.nn.Embedding(vocab_size, output_dim)

In [18]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [19]:
max_length = 4
dataLoader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataLoader)
inputs,targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [20]:
embeddings = token_embedding_layer(inputs)
print(embeddings)

tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.3163,  ..., -1.1582, -1.13

In [21]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding_layer = pos_embedding_layer(torch.arange(context_length))
print(pos_embedding_layer.shape)

torch.Size([4, 256])


In [22]:
input_embeddings = embeddings + pos_embedding_layer
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [23]:
#Working on attention
import torch
inputs = torch.tensor (
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [24]:
#Take the first input cell... which is Journey
query = inputs[1]

#Calculate attention scores
#Create an empty torch tensor of the shape of the inputs
attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    # For each word in the embedding calculate the dot product between the 
    # target word and the remaining words in the input
    # O(n^2) complexity
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)



tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [25]:
#Now we do softmax

attn_weights_2_tmp = attn_scores_2/ attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [26]:
#Now actually using softmax
def softmax_naive(x):
    return torch.exp(x)/ torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(attn_weights_2_naive)


tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [27]:
attn_weights2 = torch.softmax(attn_scores_2, dim=0)
print(attn_weights2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [28]:
#The final of the final ! We multiply the normalised attention weights with the 

query = inputs[1]

context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [29]:
#Coding attention mechanism (with trainable weights):

#1 -> the second input element (journey)
x_2 = inputs[1] 
#2 -> Input Embedding size of 3
d_in = inputs.shape[1]
#3 -> Output embedding size of 2
d_out = 2

In [30]:
torch.manual_seed(123)
#
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [31]:
#computing the Q,K and values
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)
print(key_2)

tensor([0.4306, 1.4551])
tensor([0.4433, 1.1419])


In [32]:
keys = inputs @ W_key
values = inputs @ W_value
print(keys)
print(values)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])
tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])


In [33]:
keys_2 = keys[1] #1
#The query is then used as a dot product with the key ??? Damn why ?
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [34]:
#Again the attention score is then generalized to all keys (Attention score = query dot product with all keys)
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [35]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [36]:
context_vec_2 = attn_weights_2 @ values

print(context_vec_2)

tensor([0.3061, 0.8210])


In [37]:
#Import the torch neural net as nn
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    #Initialize -----> 
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    #Forward -> k,q,v
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T #Omega
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=1)
        context_vec = attn_weights @ values
        return context_vec

In [38]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [39]:
#Import the torch neural net as nn
import torch.nn as nn
class SelfAttention_v2(nn.Module):
    #Initialize -----> 
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out,bias=qkv_bias)

    #Forward -> k,q,v
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T #Omega
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [40]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [41]:
#3.5 causal attention masking

#Step 1
#Finding the attention weights using the softmax function
queries = sa_v2.W_query(inputs) #1
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [42]:
#Step 2: We use the pytorch's tril function to create a mask where the value above the diagnol is 0
#The context length is the dimentions of the attention scores
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)
#Now we can multiply this matrix with the attention weights to get a masked attention weight matrix :)
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [43]:
#Step 3: Re normalizing the array by dividing each element by sum of that row !!!!
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple/row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [44]:
#Using a better approach by masking using -inf in step 2
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [45]:
attn_weights = torch.softmax(masked/ keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [46]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [47]:
class CausalAttention(nn.Module):
    def __init__(self,d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
           'mask',
           torch.triu(torch.ones(context_length, context_length),
           diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape                   #3
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)   
        attn_scores.masked_fill_(                    #4
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec


In [48]:
#3.6 multi head attention module 
#This is a basic stacking based module !

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([CausalAttention(
                d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [49]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)
context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [50]:
#Integrated multi attention head class :)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        #Initialize the nn.Module super class
        super().__init__()
        
        assert(d_out % num_heads == 0), \
        "d_out must be divisible by the number of heads"

        self.d_out = d_out
        
        self.num_heads = num_heads
        
        self.head_dim = d_out // num_heads
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.out_proj = nn.Linear(d_out, d_out)    #2
        
        self.dropout = nn.Dropout(dropout)
        
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )


    def forward(self, x):
        
        b, num_tokens, d_in = x.shape
        
        keys = self.W_key(x)         #3
        
        queries = self.W_query(x)    #3
        
        values = self.W_value(x)     #3

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)       #4
        
        values = values.view(b, num_tokens, self.num_heads, self.head_dim) 
        
        queries = queries.view(                                             
            b, num_tokens, self.num_heads, self.head_dim                    
        )                                                                   

        
        keys = keys.transpose(1, 2)          #5
        
        queries = queries.transpose(1, 2)    #5
        
        values = values.transpose(1, 2)      #5

        attn_scores = queries @ keys.transpose(2, 3)   #6
        
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]    #7

        attn_scores.masked_fill_(mask_bool, -torch.inf)     #8

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)   #9
 #10
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        
        context_vec = self.out_proj(context_vec)    #11
        
        return context_vec
